In [ ]:
!pip install transformers
!pip install pytorch
!pip install wandb
!pip install evaluate
!pip install torch torchvision torchaudio --pre -f https://download.pytorch.org/whl/nightly/cu111/torch_nightly.html
!pip install tqdm
!pip install drive
!pip install pytorch-lightning
#!pip install --upgrade scikit-learn
!wandb login c0e239acadd2064bd06d6d5065d786d67c616112

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
  error: subprocess-exited-with-error
  
  × Running setup.py install for pytorch did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 44.2 MB/s eta 0:00:00
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb
import transformers
import torch
import time 

import pandas as pd 
import numpy as np
from tqdm import tqdm

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

import pyarrow.parquet as pq
from google.colab import drive

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler


from transformers import BertTokenizer, BertModel ,BertConfig, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import TrainingArguments
from transformers import AdamW
from transformers import AutoModelForSequenceClassification



from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch.nn.functional as F
from torch import cuda

import torch, gc, random, datasets
from transformers.file_utils import is_tf_available, is_torch_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error,mean_absolute_percentage_error
import pandas as pd
import numpy as np
import pickle


In [ ]:
train = pq.ParquetFile("/content/drive/MyDrive/Masterarbeit/Data/train2.parquet")
df_train = train.read().to_pandas()
test = pq.ParquetFile("/content/drive/MyDrive/Masterarbeit/Data/test2.parquet")
df_test = test.read().to_pandas()
val = pq.ParquetFile("/content/drive/MyDrive/Masterarbeit/Data/val2.parquet")
df_val = val.read().to_pandas()

In [ ]:
with open('/content/drive/MyDrive/Masterarbeit/Data/pricescaler2.pkl', 'rb') as f:
    price_scaler = pickle.load(f)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.0 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
max_length=400
model_name="bert-base-multilingual-cased"

# aufgeteilte Daten laden 
X_train=df_train["title_description"]
y_train=df_train["preis_bereinigt"]
X_test=df_test["title_description"]
y_test=df_test["preis_bereinigt"]
X_val=df_val["title_description"]
y_val=df_val["preis_bereinigt"]

# Tokenizer aufrufen
tokenizer = AutoTokenizer.from_pretrained(model_name)
 
train_encodings = tokenizer(X_train.tolist(),add_special_tokens=True,padding='max_length',truncation='longest_first',max_length=max_length,return_attention_mask=True)

valid_encodings = tokenizer(X_val.tolist(),add_special_tokens=True,padding='max_length',truncation='longest_first',max_length=max_length,return_attention_mask=True)

test_encodings= tokenizer(X_test.tolist(),add_special_tokens=True,padding='max_length',truncation='longest_first',max_length=max_length,return_attention_mask=True)



class MakeTorchData(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        item["labels"] = float(item["labels"])
        return item

    def __len__(self):
        return len(self.labels)

# torch Datenset aus tokenized Daten 
train_dataset = MakeTorchData(train_encodings, y_train.ravel())
valid_dataset = MakeTorchData(valid_encodings, y_val.ravel())
test_dataset = MakeTorchData(test_encodings, y_test.ravel())

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                           num_labels = 1)#.to("cuda")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
def compute_metrics_for_regression(eval_pred):

    logits, labels = eval_pred

    labels_re= price_scaler.inverse_transform(labels.reshape(1, -1))
    logits_re= price_scaler.inverse_transform(logits.reshape(1, -1))
    

    #labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels_re, logits_re)
    #rmse = mean_squared_error(labels_re, logits_re, squared=False)
    mae = mean_absolute_error(labels_re, logits_re)
    r2 = r2_score(labels, logits)
    mape=mean_absolute_percentage_error(labels_re, logits_re)
    #smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)
    #"smape": smape
    #"rmse": rmse
    return {"mse": mse, "mae": mae, "r2": r2,"mape": mape}

In [ ]:
num_epochs=50

# Argumente für Training angeben 
training_args = TrainingArguments(
    output_dir ='/content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2',          
    num_train_epochs = num_epochs, 
    bf16=True,    
    per_device_train_batch_size = 64,   
    per_device_eval_batch_size = 64,   
    weight_decay = 0.01,               
    learning_rate = 5e-5,
    logging_dir = '/content/drive/MyDrive/Masterarbeit/BERT_pretrained/logs', 
    #Wie viele Checkpoint Files speichern...         
    save_total_limit = 10,
    load_best_model_at_end = True,     
    metric_for_best_model = 'r2',    
    evaluation_strategy = "epoch",
    #Wie oft Fortschritt training
    logging_steps=10,
    save_strategy = "epoch",
    report_to="wandb",
    #resume_from_checkpoint="/content/drive/MyDrive/Masterarbeit/BERT_pretrained/checkpoint-37320"

) 

"steps"

# Trainer aufrufen 
trainer = Trainer(
    model = model,                         
    args = training_args,                  
    train_dataset = train_dataset,         
    eval_dataset = valid_dataset,          
    compute_metrics = compute_metrics_for_regression,    
)

# Modell trainieren 
trainer.train()




Using cuda_amp half precision backend
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 199006
  Num Epochs = 50
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 155500
  Number of trainable parameters = 177854209
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: natascha-ris (masterarbeitnri). Use `wandb login --relogin` to f

Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Mape
1,0.005800,0.006417,108536528896.000000,237826.390625,0.701254,0.459762
2,0.005400,0.005518,93336076288.000000,207635.765625,0.743093,0.341086
3,0.005000,0.006142,103883972608.000000,232993.671875,0.714060,0.435098
4,0.003800,0.004968,84023312384.000000,200295.203125,0.768727,0.357359
5,0.002800,0.005020,84915781632.000000,201026.125000,0.766270,0.351063
6,0.002800,0.004811,81376108544.000000,186382.671875,0.776013,0.272827
7,0.002300,0.004854,82093465600.000000,181590.140625,0.774039,0.268373
8,0.001900,0.005209,88110874624.000000,196910.984375,0.757476,0.318605
9,0.001900,0.004468,75580203008.000000,174079.296875,0.791966,0.268069
10,0.001600,0.004430,74937647104.000000,179207.515625,0.793735,0.296482


***** Running Evaluation *****
  Num examples = 24876
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2/checkpoint-3110
Configuration saved in /content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2/checkpoint-3110/config.json
Model weights saved in /content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2/checkpoint-3110/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 24876
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2/checkpoint-6220
Configuration saved in /content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2/checkpoint-6220/config.json
Model weights saved in /content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2/checkpoint-6220/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 24876
  Batch size = 64
Saving model checkpoint to /content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2/checkpoint-9330
Configuratio

TrainOutput(global_step=155500, training_loss=0.0010618450669874961, metrics={'train_runtime': 47082.3751, 'train_samples_per_second': 211.338, 'train_steps_per_second': 3.303, 'total_flos': 2.0453206467799178e+18, 'train_loss': 0.0010618450669874961, 'epoch': 50.0})

In [ ]:
# Zusammenfassung der Validierungsdaten 
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 24876
  Batch size = 64


{'eval_loss': 0.003640628419816494,
 'eval_mse': 61578510336.0,
 'eval_mae': 152578.046875,
 'eval_r2': 0.8305057775373601,
 'eval_mape': 0.22247321903705597,
 'eval_runtime': 35.1103,
 'eval_samples_per_second': 708.51,
 'eval_steps_per_second': 11.079,
 'epoch': 50.0}

In [ ]:
#Es wurde MAPE Wert falsch berechnet, deshalb wird unten die Evaluierung wiederholt... 
trainer.eval_dataset=test_dataset
# Zusammenfassung 
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 24876
  Batch size = 64


{'eval_loss': 0.003699674503877759,
 'eval_mse': 62577229824.0,
 'eval_mae': 153355.96875,
 'eval_r2': 0.8282444502860519,
 'eval_mape': 9370335232.0,
 'eval_runtime': 35.3211,
 'eval_samples_per_second': 704.281,
 'eval_steps_per_second': 11.013,
 'epoch': 50.0}

#Evaluierung Testdatensatz

In [ ]:
model_name="bert-base-multilingual-cased"
#model_name="bert-base-multilingual-cased"
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2/checkpoint-149280')
#model = AutoModelForSequenceClassification.from_pretrained('models/checkpoint-5394')
model.eval()
# Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Trainer
training_args = TrainingArguments(
    output_dir ='/content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2',              
    per_device_train_batch_size = 16,   
    per_device_eval_batch_size = 16,   
    weight_decay = 0.01,               
    learning_rate = 3e-5,
    logging_dir = '/content/drive/MyDrive/Masterarbeit/BERT_pretrained/logs',          
    save_total_limit = 10,
    load_best_model_at_end = True,     
    metric_for_best_model = 'r2',    
    evaluation_strategy = "epoch",
    logging_steps=10,
    save_strategy = "epoch",
    report_to="wandb",
      resume_from_checkpoint='/content/drive/MyDrive/Masterarbeit/BERT_pretrained/BERTNEW2/checkpoint-149280',
) 


trainer = Trainer(
    model = model,                         
    args = training_args,                       
    eval_dataset = test_dataset,          
    compute_metrics = compute_metrics_for_regression,    
  
)

In [ ]:
trainer.eval_dataset=test_dataset
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 24876
  Batch size = 16


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: natascha-ris (masterarbeitnri). Use `wandb login --relogin` to force relogin


{'eval_loss': 0.0036999669391661882,
 'eval_mse': 62582173696.0,
 'eval_mae': 153373.5625,
 'eval_r2': 0.8282308857235272,
 'eval_mape': 0.16954144835472107,
 'eval_runtime': 606.7565,
 'eval_samples_per_second': 40.998,
 'eval_steps_per_second': 2.563}